### Goals

1. Add more Tools / Agents to enhance capabilities
   - Add Another Tool for booking
2. Add an Agent that translates all responses to a different language and shows on the right side, try with different Frontier Model
3. Add and Agent that can listen for Audio and convert it to Text

# Add More Tools

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

In [5]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [6]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [7]:
### Price Tool

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
### Handle Price Tool

def handle_price_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [9]:
### Booking Tool

def book_flight():
    print("Ticket has been booked")
    return {'status': 200}

book_function = {
    "name": "book_flight",
    "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book tihs flight?'",
    "parameters": {}
}

In [10]:
def handle_book_flight_call(message):
    tool_call = message.tool_calls[0]
    status = book_flight()
    response = {
        "role": "tool",
        "content": json.dumps({"status": "Ticket has been booked"}),
        "tool_call_id": tool_call.id
    }
    return response, status

In [11]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": book_function}]

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        if response.choices[0].message.tool_calls[0].function.name=="get_ticket_price":
            message = response.choices[0].message
            response, city = handle_price_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        elif response.choices[0].message.tool_calls[0].function.name=="book_flight":
            message = response.choices[0].message
            response, status = handle_book_flight_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [13]:
chat("Great, can you help me book the flight?", 
    [
        {"role": "user", "content": "Hi, how's it going?"}, 
        {"role": "assistant", "content": "Hello! I'm here to help you with your travel needs. How can I assist you today?"},
        {"role": "user", "content": "How about Paris?"}, 
        {"role": "assistant", "content": "Could you please specify what information you would like about Paris?"},
        {"role": "user", "content": "What are things to see there?"}, 
        {"role": "assistant", "content": "In Paris, you can explore the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and enjoy a stroll along the Seine River."},
        {"role": "user", "content": "Can you tell me the ticket price for a flight to Paris?"}, 
        {"role": "assistant", "content": "The ticket price for a flight to Paris is approximately $899."}        
    ]
)

Ticket has been booked


'Your flight to Paris has been successfully booked!'

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Add Tool Functionality with Cluade

In [12]:
import os
import json
from dotenv import load_dotenv
import anthropic
import gradio as gr

In [14]:
load_dotenv(override=True)

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL = "claude-3-7-sonnet-latest"
client = anthropic.Anthropic(api_key=anthropic_api_key)

In [60]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "The user will send all messages in English. Respond only in Spanish."

In [33]:
## Antropic version of chat function
history_store = []

def chat(message, history):
    global history_store

    # Convert Gradio's history format to Anthropic format
    messages = []
    for turn in history_store:
        messages.append({"role": "user", "content": turn["user"]})
        messages.append({"role": "assistant", "content": turn["assistant"]})

    messages.append({"role": "user", "content": message})
    
    response = client.messages.create(system=system_message, model=MODEL, max_tokens=1024, messages=messages)

    assistant_response = ""
    
    for block in response.content:
        if block.type == "text":
            assistant_response += block.text    

    history_store.append({"user": message, "assistant": assistant_response})
    
    return assistant_response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [ ]:
## With Tools
tools = [
    {
        "name": "get_ticket_price",
        "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to"
                }
            },
            "required": ["destination_city"]
        }
    },
    {
        "name": "book_flight",
        "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book this flight?'",
        "input_schema": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city that the customer wants to travel to"
                }
            },
            "required": ["destination_city"]
        }
    }    
]

def execute_tool(name, input_data):
    if name == "get_ticket_price":
        ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
        city = input_data.get("destination_city", "Unknown City")
        print(f"Tool get_ticket_price called for {city}")
        city = city.lower()
        price = ticket_prices.get(city, "Unknown")
        # return {"destination_city": city,"price": price}
        return f"The ticket for a flight to {city} is {price}"
    elif name == "book_flight":
        print("Ticket has been booked")
        # return {"ticket_status": "Successfully booked"}
        return "Ticket has been booked"

history_store = []

def chat(message, history):
    global history_store

    # Convert Gradio's history format to Anthropic format
    messages = []
    for turn in history_store:
        messages.append({"role": "user", "content": turn["user"]})
        messages.append({"role": "assistant", "content": turn["assistant"]})

    messages.append({"role": "user", "content": message})
    
    response = client.messages.create(system=system_message, model=MODEL, max_tokens=1024, tools=tools, messages=messages)

    assistant_response = ""
    tool_call = None
    
    for block in response.content:
        if block.type == "text":
            assistant_response += block.text   
        elif block.type == "tool_use":
            tool_call = block        

   # If tool is requested
    if tool_call:
        tool_result_data = execute_tool(tool_call.name, tool_call.input)

        # Send tool output back to Claude
        tool_followup = client.messages.create(
            model=MODEL,
            system=system_message,
            max_tokens=1024,
            messages=[
                *messages,
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "tool_use",
                            "id": tool_call.id,
                            "name": tool_call.name,
                            "input": tool_call.input
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_call.id,
                            "content": tool_result_data
                        }
                    ]
                }
            ]
        )
        
        assistant_response = "".join(
            block.text for block in tool_followup.content if block.type == "text"
        )        

    history_store.append({"user": message, "assistant": assistant_response})
    
    return assistant_response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7897
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Add translation functionality

Here, what I want to do is very simply copy the same functionality from OpenAI's chat, but just add an additional part which also translates and just adds the translation to a new line. Example below:

```
import gradio as gr
from transformers import pipeline

# Initialize translation pipeline (English to French)
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

# Main chatbot response function
def respond(message, history):
    # Simulate LLM response (you can replace this with OpenAI or other LLM API call)
    llm_response = f"You said: {message}"

    # Translate the response to French
    translation = translator(llm_response, max_length=100)[0]['translation_text']

    # Combine original and translated response
    full_response = f"{llm_response}\n\n_Translated:_ {translation}"

    return full_response

# Gradio ChatInterface
chat = gr.ChatInterface(fn=respond, 
                        title="Bilingual Chat",
                        description="This chatbot replies and also translates its responses to French.")

chat.launch()
```


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

In [11]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [33]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "You are also trying to teach Spanish. The user will se sending their english and spanish translation."
system_message += "Always reply back with an English and Spanish version of the response like this:"
system_message += "Response: {English response}"
system_message += "Translated: {Spanish response}"

In [34]:
### Price Tool

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [35]:
### Handle Price Tool

def handle_price_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [36]:
### Booking Tool

def book_flight():
    print("Ticket has been booked")
    return {'status': 200}

book_function = {
    "name": "book_flight",
    "description": "Book a flight for a destination. Call this whenever the user requests to book the flight, for example when a customer asks 'Can you help me book tihs flight?'",
    "parameters": {}
}

In [37]:
def handle_book_flight_call(message):
    tool_call = message.tool_calls[0]
    status = book_flight()
    response = {
        "role": "tool",
        "content": json.dumps({"status": "Ticket has been booked"}),
        "tool_call_id": tool_call.id
    }
    return response, status

In [38]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": book_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        if response.choices[0].message.tool_calls[0].function.name=="get_ticket_price":
            message = response.choices[0].message
            response, city = handle_price_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        elif response.choices[0].message.tool_calls[0].function.name=="book_flight":
            message = response.choices[0].message 
            response, status = handle_book_flight_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Ticket has been booked


# Agent to listen for Audio and convert to Text